In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from mongo import attraction_db
from model.google import Google
from model.Word import Word
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger
import random
import os
import json
import time
import csv
import os

In [ ]:
url = 'https://www.google.com/maps/search/%E6%99%AF%E9%BB%9E'

driver = webdriver.Chrome()
driver.get(url)

In [ ]:
def getPage():
    page_content = driver.page_source
    # 將 HTML 內容轉換成 BeautifulSoup 物件，html.parser 為使用的解析器
    soup = BeautifulSoup(page_content, 'html.parser')
    # 透過 select 使用 CSS 選擇器 選取我們要選的 html 內容
    return soup.select('.Nv2PK.THOPZb.CpccDe')

def scrollPage(times):   # 滾動頁面
    counter = 0
    while counter <= times:
        pane = driver.find_element("xpath",'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]')
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)
        counter += 1

def getElement():  # 抓取目前的所有留言
    spots_list = []
    scrollPage(5)
    elements = getPage()

    for element in elements:
        name = element.select('.qBF1Pd.fontHeadlineSmall')[0].text
        url = element.select('.hfpxzc')[0].get('href')
        dataset = {}
        dataset['name'] = name
        dataset['google_url'] = url

        spots_list.append(dataset)
    return spots_list

In [ ]:
lastSpot = ''
while 7 == 7:
    print(lastSpot)
    spots_list = getElement()
    if (spots_list[len(spots_list)-1]["name"] == lastSpot ):
        break
    lastSpot = spots_list[len(spots_list)-1]["name"]
    time.sleep(1)
    getElement()

csv_file = './data/spot_list/spots.csv'
if os.path.exists(csv_file):
    with open(csv_file, mode="r", newline="", encoding="utf-8-sig") as file:
        reader = csv.DictReader(file)
        existing_data = list(reader)
else:
    existing_data = []

for spot in spots_list:
    existing_data.append(spot)

print(len(existing_data))


with open(csv_file, mode="w", newline="", encoding="utf-8-sig") as file:
    # 定義 CSV 欄位
    fieldnames = ["name", "google_url"]
    # 建立 CSV 寫入器
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(existing_data)

    print(f"已將資料寫入 CSV 檔案：{csv_file}")

In [ ]:
dataset = pd.read_csv('./data/spot_list/spotsList_clean.csv')
dataset

In [ ]:
df_unique = dataset.drop_duplicates(subset="name", keep="first")
df_unique.to_csv('./data/spot_list/spotsList.csv', index=False, encoding="utf-8-sig")
df_unique

In [ ]:
existing_names = attraction_db.attractionInfo.distinct("name")
df_filtered = df_unique[~df_unique["name"].isin(existing_names)]
df_filtered.to_csv('./data/spot_list/spotsList.csv', index=False, encoding="utf-8-sig")

In [ ]:
existing_names = attraction_db.attractionInfo.distinct("name")
df = pd.read_csv('./data/spot_list/random_spotsList.csv')
df_filtered = df[~df["name"].isin(existing_names)]
df_filtered.to_csv('./data/spot_list/spotsList_clean.csv', index=False, encoding="utf-8-sig")
df_filtered

In [ ]:
# ============================================
# TEST
# ============================================

spotsList = pd.read_csv('./data/spot_list/spotsList_clean.csv') 

for index, row in spotsList[558:570].iterrows():
    print(index,row['name'],row['google_url'])

In [ ]:
# 取得 Google URL

name_to_url = {}
with open('./data/spot_list/spotsList_clean.csv', 'r', encoding='utf-8') as spots_file:
    spots_reader = csv.reader(spots_file)
    next(spots_reader)  # 跳过标题行
    for row in spots_reader:
        name = row[0]  # 假设名称在第一列
        url = row[1]   # 假设URL在第二列
        name_to_url[name] = url

def get_url_for_name(name):
    return name_to_url.get(name, "URL not found")

In [ ]:
# 含有多少沒有抓

existing_names = attraction_db.attractionInfo.distinct("name")
df = pd.read_csv('./data/spot_list/spotsList_clean.csv')
df_filtered = df[~df["name"].isin(existing_names)]
df_filtered

In [ ]:
# ============================================
# 個別處理
# ============================================

name = '講美鎮海社區海堤公園'
url  = get_url_for_name(name)

data = pd.read_csv('./data/tfidf_csv/{}.csv'.format(name))
tag_value = data['words'].tolist()

place_detail_data = Google().get_place_info(name,url,tag_value)
place_detail_data = json.loads(place_detail_data)
print(place_detail_data)

existing_data = attraction_db.attractionInfo.find_one({
    "$or": [
        {"place_id": place_detail_data["place_id"]},
        {"id": place_detail_data["id"]}
    ]
})
print(existing_data)
if existing_data is None:
    attraction_db.attractionInfo.insert_one(place_detail_data)
    print(f"{place_detail_data['name']}：已新增")

In [ ]:
# 檢查未成功插入資料庫的景點

existing_names = set(doc['name'] for doc in attraction_db.attractionInfo.find({}, {"name": 1}))

csv_folder_path = './data/tfidf_csv/' 
csv_files = os.listdir(csv_folder_path)
new_csv_files = [csv_file.split('.')[0] for csv_file in csv_files if csv_file.split('.')[0] not in existing_names]
print(len(new_csv_files))
print(new_csv_files)

In [ ]:
# 新增缺少的景點
wrong = []
# new_csv_files = ['三峽螢火蟲秘境']
for spot in new_csv_files:
    name = spot
    url = get_url_for_name(name)
    try:
        try:
            tfidf = pd.read_csv('./data/tfidf_csv/{}.csv'.format(name))
            tag_value = tfidf['words'].tolist()
        except:
            tag_value = []
        
        place_detail_data = Google().get_place_info(name,url,tag_value)
        place_detail_data = json.loads(place_detail_data)
        existing_data = attraction_db.attractionInfo.find_one({
            "$or": [
                {"place_id": place_detail_data["place_id"]},
                {"id": place_detail_data["id"]}
            ]
        })
        if existing_data is None:
            attraction_db.attractionInfo.insert_one(place_detail_data)
            print(f"{place_detail_data['name']}：已新增")
        else:
            wrong.append({
                "name":place_detail_data['name'],
                "existing_data":existing_data['name']
            })
            print(f"{place_detail_data['name']}：未新增",existing_data['name'])
    except Exception as e:
        wrong.append({
            "name":name,
            "existing_data":e
        })
        print(f"錯誤訊息：{e}")
        continue

print(wrong)
with open("./data/warn/check.json", "w", encoding="utf8") as f:
    json.dump(wrong, f, indent=2, sort_keys=True, ensure_ascii=False)

In [ ]:
# 刪除不必要的景點
for spot in new_csv_files:
    name = spot
    csv_file_path = './data/tfidf_csv/{}.csv'.format(name)
    if os.path.exists(csv_file_path):
        os.remove(csv_file_path)
        print(f"{name}.csv 已删除")

In [ ]:
# 隨機排序景點

list = pd.read_csv('./data/spot_list/spotsList.csv')
print(list)

random.seed(42)
random_spotsList = list.sample(frac=1,random_state=random.randint(1,10000))
print(random_spotsList)

random_spotsList.to_csv('./data/spot_list/random_spotsList.csv',encoding="utf-8-sig",index=False)


In [ ]:
# ============================================
# 個別處理 2
# ============================================
ws_driver = CkipWordSegmenter(model="bert-base")
pos_driver = CkipPosTagger(model="bert-base")

In [ ]:
csv_folder_path = './data/set/' 
existing_names = attraction_db.attractionInfo.distinct("name")
csv_files = os.listdir(csv_folder_path)
tfidf_files = [csv_file.split('.')[0] for csv_file in csv_files if csv_file.split('.')[0] not in existing_names]
tfidf_files

In [ ]:
for spot in tfidf_files:
    name = spot
    url = get_url_for_name(name)
    try:
        dataset = pd.read_excel('./data/set/{}.xlsx'.format(name))
        tag = Word(ws_driver,pos_driver).save(dataset)
        tag_value = tag['words'].tolist()
        
        place_detail_data = Google().get_place_info(name,url,tag_value)
        place_detail_data = json.loads(place_detail_data)
        existing_data = attraction_db.attractionInfo.find_one({
            "$or": [
                {"place_id": place_detail_data["place_id"]},
                {"id": place_detail_data["id"]}
            ]
        })
        if existing_data is None:
            attraction_db.attractionInfo.insert_one(place_detail_data)
            print(f"{place_detail_data['name']}：已新增")
        else:
            print(f"{place_detail_data['name']}：未新增")
    except Exception as e:
        print(f"錯誤訊息：{e}")
        continue